# 1. Models & Tokenizers

In [1]:
!pip install --q transformers

In [2]:
from transformers import AutoTokenizer

In [3]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
tokenizer("Hello World")

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [7]:
tokens = tokenizer.tokenize("Hello World")
tokens

['hello', 'world']

In [9]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[7592, 2088]

In [10]:
tokenizer.convert_ids_to_tokens(ids)

['hello', 'world']

In [11]:
tokenizer.decode(ids)

'hello world'

In [12]:
ids = tokenizer.encode("hello world")
ids

[101, 7592, 2088, 102]

In [13]:
tokenizer.convert_ids_to_tokens(ids)

['[CLS]', 'hello', 'world', '[SEP]']

We see the tokenizer adds two BERT special tokens: `[CLS]` and `[SEP]`

In [14]:
tokenizer.decode(ids)

'[CLS] hello world [SEP]'

In [15]:
model_inputs = tokenizer("hello world")
model_inputs

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [17]:
# Tokenize multiple sentences at once:
data= [
    "I like cats.",
    "Do you like cats too?",
]
tokenizer(data)

{'input_ids': [[101, 1045, 2066, 8870, 1012, 102], [101, 2079, 2017, 2066, 8870, 2205, 1029, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [18]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model_inputs = tokenizer("hello world", return_tensors='pt')

outputs = model(**model_inputs) # as using a PyTorch model, it just accepts torch tensors as inputs
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[0.4758, 0.5030]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

The default was to create a binary classifier!

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3364,  0.5550, -0.1813]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## Few ways we can access logits

In [24]:
outputs.logits

tensor([[-0.3364,  0.5550, -0.1813]], grad_fn=<AddmmBackward0>)

In [25]:
outputs['logits']

tensor([[-0.3364,  0.5550, -0.1813]], grad_fn=<AddmmBackward0>)

In [27]:
outputs.logits.detach().cpu().numpy() # If we wanted to comput logits into numpy array, e.g. for acc or auc computation

array([[-0.33635485,  0.55498725, -0.18131647]], dtype=float32)

## Back to demonstrating how to process multiple strings at once

In [29]:
data = [
  "I like cats.",
  "Do you like cats too?",
]
model_inputs = tokenizer(data, padding=True, truncation=True, return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [30]:
model_inputs['input_ids']

tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]])

In [31]:
model_inputs['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])

In [32]:
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1097,  0.5827, -0.3432],
        [-0.0794,  0.5943, -0.3126]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

**Note:** 2x3 tensor as an output. This makes sense, as we passed two docs as an input, and the number of classes is 3.